In [1]:
import xcast as xc 
import xarray as xr 
import cartopy.crs as ccrs 
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from pathlib import Path
import os
import time
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import glob

In [2]:
ddir='/cpc/int_desk/pac_isl/analysis/xcast/seasonal/practical_notebooks/practical_data'
fdir='/cpc/int_desk/pac_isl/analysis/xcast/seasonal/presentation_figures'

In [3]:
obs_name='CMORPH'

In [4]:
initial_dates = [(2024, 6, 1)]

In [5]:
pacislands_coordinates = {
    'west': 130,
    'east': 205,
    'north': 8,
    'south': -20
    }

chuuk_coordinates = {
    'west': 151,
    'east': 153,
    'north': 8,
    'south': 6
    }

fiji_coordinates = {
    'west':  177,
    'east': 182,  
    'north': -15,  
    'south': -20}

kiribati_coordinates = {
        'west':  202,
        'east': 203,  
        'north': 2.5,  
        'south': 1}

solomon_coordinates = {
        'west':  155,
        'east': 167,  
        'north': -6,  
        'south': -13}

png_coordinates = {
        'west':  130,
        'east': 156,  
        'north': 1,  
        'south': -12}
        
palau_coordinates = {
        'west':  133,
        'east': 135,  
        'north': 8,  
        'south': 6
}
        
vanuatu_coordinates = {
        'west':  165,
        'east': 170,  
        'north': -12.5,  
        'south': -20
}

samoa_coordinates = {
        'west':  187,
        'east': 191,  
        'north': -13,  
        'south': -15
}

tuvalu_coordinates = {
        'west':  179,
        'east': 180,  
        'north': -8,  
        'south': -9
}

#Pacific region, encompassing all islands
pacific_extent = {
    'west': 120,
    'east': 210,
    'north': 10,
    'south': -30
}

predictor_train_extent = pacific_extent
predictor_train_extent_name = 'pacific'
regions = [vanuatu_coordinates]
region_names = ['Vanuatu']

#month dictionary
number_to_month_name_dictionary = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec',
    0: 'Dec'
}

In [6]:
initial_months, initial_month_names, target_seasons, target_months = [], [], [], []
 
for i in initial_dates:
    leads = [['1', '3'],['2', '4'], ['3','5']]
    initial_month = dt.datetime(*i).month
    initial_months.append(initial_month)
    initial_month_names.append(number_to_month_name_dictionary[initial_month])
    target_month = []
    target_seas = []
    for l in leads:
        target_low = number_to_month_name_dictionary[(initial_month + float(l[0]))%12]
        target_mid = number_to_month_name_dictionary[(initial_month + float(l[0])+1)%12]
        target_high = number_to_month_name_dictionary[(initial_month + float(l[1]))%12]
        target_seas.append('-'.join([target_low, target_high]))
        target_month.append(target_low[0] + target_mid[0] + target_high[0])
    target_seasons.append(target_seas)
    target_months.append(target_month)

In [11]:
for t, initial_month_name in enumerate(initial_month_names):
    target_seas = target_seasons[t]
    ###### read in observations
    if obs_name == 'CMORPH':
        training_length = 'threeseas'
    elif obs_name == 'CHIRPS':
        training_length = 'oneseas'
    obs_leads = xr.open_dataset(os.path.join(ddir, '_'.join([initial_month_name, training_length, obs_name, 'precip.nc'])))

    ###### read in hindcast and forecast data
    hindcast_data_precip = xr.open_dataset(os.path.join(ddir,
    '_'.join([initial_month_name, training_length, 'NMME_hcst_precip.nc'])))
    forecast_data_precip = xr.open_dataset(os.path.join(ddir, '_'.join([initial_month_name, training_length, 'NMME_fcst_precip.nc'])))  
    
    hindcast_data_sst = xr.open_dataset(os.path.join(ddir,
    '_'.join([initial_month_name, training_length, 'NMME_hcst_sst.nc'])))
    forecast_data_sst = xr.open_dataset(os.path.join(ddir, '_'.join([initial_month_name, training_length, 'NMME_fcst_sst.nc'])))  
    
    ###### read in the ocean mask for the data
    msk = xr.open_dataset('/cpc/africawrf/ebekele/projects/PREPARE_pacific/notebooks/masked/libs/pacific_mask.nc')
    mskk = msk.amask.expand_dims({'M':[0]})
    mskk = mskk.assign_coords({'lon': [i + 360 if i <= 0 else i for i in mskk.coords['lon'].values]}).sortby('lon').drop_duplicates('lon')
    mskk = mskk.rename({'lon':'X', 'lat':'Y', 'time':'T'})
    mskk = xc.regrid(mskk, obs_leads.X, obs_leads.Y)
    mask_missing = mskk.mean('T', skipna=False).mean('M', skipna=False)
    mask_missing = xr.ones_like(mask_missing).where(~np.isnan(mask_missing), other=np.nan )

In [12]:
hindcast_data_sst

<xarray.Dataset>
Dimensions:  (M: 1, Y: 61, X: 360, T: 75, L: 3)
Coordinates:
  * M        (M) object 'NMME'
  * Y        (Y) float32 -30.0 -29.0 -28.0 -27.0 -26.0 ... 27.0 28.0 29.0 30.0
  * X        (X) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * T        (T) datetime64[ns] 2024-06-27 2025-06-27 ... 2097-06-27 2098-06-27
  * L        (L) int64 1 2 3
Data variables:
    sst      (L, M, T, Y, X) float64 ...

In [22]:
#NMME precip data
hindcast_data_precip = hindcast_data_precip.sel(X=slice(predictor_train_extent['west'], predictor_train_extent['east']), Y=slice(predictor_train_extent['south'], predictor_train_extent['north']))
forecast_data_precip = forecast_data_precip.sel(X=slice(predictor_train_extent['west'], predictor_train_extent['east']), Y=slice(predictor_train_extent['south'], predictor_train_extent['north']))

#NMME sst data
hindcast_data_sst = hindcast_data_sst.sel(X=slice(predictor_train_extent['west'], predictor_train_extent['east']), Y=slice(predictor_train_extent['south'], predictor_train_extent['north']))
forecast_data_sst = forecast_data_sst.sel(X=slice(predictor_train_extent['west'], predictor_train_extent['east']), Y=slice(predictor_train_extent['south'], predictor_train_extent['north']))

In [26]:
for r, region in enumerate(regions):
        #expand the training area to help create more to train for smaller islands
        predictand_train_extent = {
            'west':  region['west']-5,
                'east': region['east']+5,  
                'north': region['north']+3,  
                'south': region['south']-2
        }
        
        #chuuk fell out of ocean mask, workaround below
        if region == chuuk_coordinates:
            obs_leads = obs_leads.copy()
        else:
            obs_leads = obs_leads * mask_missing
        
        #crop the observations to your training region of choice
        obs_leads = obs_leads.sel(X=slice(predictand_train_extent['west'], predictand_train_extent['east']), Y=slice(predictand_train_extent['south'], predictand_train_extent['north']))

In [27]:
##create ELR and EPOELM forecasts
start_time = time.time()
elr_fcsts_prob, elr_fcsts_det, elr_hcasts_det, elr_hcasts_prob = [], [], [], []
epoelm_fcsts_prob, epoelm_fcsts_det, epoelm_hcasts_det, epoelm_hcasts_prob = [], [], [], []
obs_to_test_grid, raw_to_test_grid = [],[]

for l in np.unique(hindcast_data_precip.L):
    obs = obs_leads.sel(L=l).precip
    model = hindcast_data_precip.sel(L=l).precip
    fmodel = forecast_data_precip.sel(L=l).precip

    model_regrid = xc.regrid(model, obs.X, obs.Y)
    fmodel_regrid = xc.regrid(fmodel, obs.X, obs.Y)

    obs, model_regrid = xc.match(obs, model_regrid)

    #run ELR, EPOELM
    hindcasts_det_ELR, hindcasts_prob_ELR, hindcasts_det_EPOELM, hindcasts_prob_EPOELM, obs_test_grid, raw_test_grid = [], [], [], [], [], []
    i=1
    for xtrain, ytrain, xtest, ytest in xc.CrossValidator(model_regrid, obs, window=5):
        print("window {}".format(i))
        i += 1
        reg_ELR = xc.ELR()
        reg_ELR.fit(xtrain, ytrain)

        reg_EPOELM = xc.EPOELM()
        reg_EPOELM.fit(xtrain, ytrain)

        preds_ELR = reg_ELR.predict(xtest)
        probs_ELR =  reg_ELR.predict_proba(xtest)
        preds_EPOELM = reg_EPOELM.predict(xtest)
        probs_EPOELM =  reg_EPOELM.predict_proba(xtest)

        obs_test_grid.append(ytest)
        raw_test_grid.append(xtest)
        hindcasts_det_ELR.append(preds_ELR)
        hindcasts_prob_ELR.append(probs_ELR)
        hindcasts_det_EPOELM.append(preds_EPOELM)
        hindcasts_prob_EPOELM.append(probs_EPOELM)
    hindcasts_det_ELR = xr.concat(hindcasts_det_ELR, 'T')
    hindcasts_prob_ELR = xr.concat(hindcasts_prob_ELR, 'T')
    hindcasts_det_EPOELM = xr.concat(hindcasts_det_EPOELM, 'T')
    hindcasts_prob_EPOELM = xr.concat(hindcasts_prob_EPOELM, 'T')
    obs_test_grid = xr.concat(obs_test_grid, 'T')
    raw_test_grid = xr.concat(raw_test_grid, 'T')

    fprobs_ELR =  reg_ELR.predict_proba(fmodel_regrid)
    fprobs_EPOELM =  reg_EPOELM.predict_proba(fmodel_regrid)

    elr_fcsts_prob.append(fprobs_ELR)
    elr_hcasts_det.append(hindcasts_det_ELR)
    elr_hcasts_prob.append(hindcasts_prob_ELR)
    epoelm_fcsts_prob.append(fprobs_EPOELM)
    epoelm_hcasts_det.append(hindcasts_det_EPOELM)
    epoelm_hcasts_prob.append(hindcasts_prob_EPOELM)
    obs_to_test_grid.append(obs_test_grid)
    raw_to_test_grid.append(raw_test_grid)

elr_fcsts_prob = xr.concat(elr_fcsts_prob, dim = 'L')
elr_hcasts_det = xr.concat(elr_hcasts_det, dim = 'L')
elr_hcasts_prob = xr.concat(elr_hcasts_prob, dim = 'L')
epoelm_fcsts_prob = xr.concat(epoelm_fcsts_prob, dim = 'L')
epoelm_hcasts_det = xr.concat(epoelm_hcasts_det, dim = 'L')
epoelm_hcasts_prob = xr.concat(epoelm_hcasts_prob, dim = 'L')
obs_to_test_grid = xr.concat(obs_to_test_grid, dim = 'L')
raw_to_test_grid = xr.concat(raw_to_test_grid, dim = 'L')
print('elr/epoelm processing time is ' + str(time.time() - start_time))

window 1


TypeError: _quantile_dispatcher() got an unexpected keyword argument 'method'